Алгоритм Дейкстры

In [222]:
input_ = '4 8\n\
1 2 6\n\
1 3 2\n\
1 4 10\n\
2 4 4\n\
3 1 5\n\
3 2 3\n\
3 4 8\n\
4 2 1\n\
1 4'

In [212]:
input_ = '4 4\n\
1 2 6\n\
1 3 2\n\
3 1 5\n\
4 3 1\n\
1 4'

In [223]:
print(input_)

4 8
1 2 6
1 3 2
1 4 10
2 4 4
3 1 5
3 2 3
3 4 8
4 2 1
1 4


In [224]:
import numpy as np
import numpy.ma as ma

In [225]:
def prepare_data(input_):
    lines = input_.split('\n')
    n_v, n_e = map(int, lines[0].split(' '))
    vertices, edges = set(), np.zeros((n_v, n_v))
    for i in range(1, n_e + 1):
        vertice_from, vertice_to, weight = map(int, lines[i].strip().split(' '))
        vertices.add(vertice_from); vertices.add(vertice_to)
        edges[vertice_from - 1, vertice_to - 1] = weight
    from_, to = map(int, lines[-1].split(' '))
    return list(vertices), edges, from_, to

In [226]:
vertices, edges, v_from, v_to = prepare_data(input_)

In [227]:
vertices

[1, 2, 3, 4]

In [228]:
edges

array([[ 0.,  6.,  2., 10.],
       [ 0.,  0.,  0.,  4.],
       [ 5.,  3.,  0.,  8.],
       [ 0.,  1.,  0.,  0.]])

In [229]:
def find_route(vertices, edges, v_from):
    visited = np.array([False] * len(vertices)) # true = visited vert
    routes_w = ma.array([np.float('inf')] * len(vertices), mask=visited)
    routes_w[v_from-1] = 0
    
    while not visited.all():
        if routes_w.min() == np.float('inf'):
            return routes_w.data
        cur_vert = routes_w.argmin() # номер минимального непосещенного узла
        for v, weight in enumerate(edges[cur_vert]):
            if weight: # exclude zero-edges: they don't exist
                if routes_w[v] > routes_w[cur_vert] + weight:
                    routes_w[v] = routes_w[cur_vert] + weight
        visited[cur_vert] = True # mark visited
        routes_w.mask = visited # change mask of masked array

    return routes_w.data

In [230]:
routes_w = find_route(vertices, edges, v_from)
routes_w

array([0., 5., 2., 9.])

In [232]:
print(int(routes_w[v_to-1])) if routes_w[v_to-1] != np.float('inf') else print(-1)

9


In [237]:
def print_distance(routes_w, v_to):
    # distance = routes_w[v_to-1]
    if v_to-1 >= len(routes_w) or routes_w[v_to-1] == np.float('inf'):
        print(-1)
    else:
        print(int(routes_w[v_to-1]))

In [239]:
print_distance(routes_w, 5)

-1


MapReduce поиск кратчайшего пути

In [1]:
input_ = '1	0	{2,3,4}\n\
2	1	{5,6}\n\
3	1	{}\n\
4	1	{7,8}\n\
5	INF	{9,10}\n\
6	INF	{}\n\
7	INF	{}\n\
8	INF	{}\n\
9	INF	{}\n\
10	INF	{}'

In [2]:
print(input_)

1	0	{2,3,4}
2	1	{5,6}
3	1	{}
4	1	{7,8}
5	INF	{9,10}
6	INF	{}
7	INF	{}
8	INF	{}
9	INF	{}
10	INF	{}


In [3]:
tmp = '{2,3,4}'

In [10]:
tmp.strip('{}').split(',')

['2', '3', '4']

In [21]:
class MapperGraph():
    def map(self, input_):
        v, d, childs = input_.strip().split('\t')
        print(input_.strip())
        for ch in childs.strip('{}').split(','):
            if ch:
                if d != 'INF':
                    print(ch, int(d) + 1, '{}', sep='\t')
                else:
                    print(ch, 'INF', '{}', sep='\t')

In [22]:
mapper = MapperGraph()
for line in input_.strip().split('\n'):
    mapper.map(line)

1	0	{2,3,4}
2	1	{}
3	1	{}
4	1	{}
2	1	{5,6}
5	2	{}
6	2	{}
3	1	{}
4	1	{7,8}
7	2	{}
8	2	{}
5	INF	{9,10}
9	INF	{}
10	INF	{}
6	INF	{}
7	INF	{}
8	INF	{}
9	INF	{}
10	INF	{}


In [23]:
input_ = '1	0	{2,3,4}\n\
10	INF	{}\n\
10	INF	{}\n\
2	1	{}\n\
2	1	{5,6}\n\
3	1	{}\n\
3	1	{}\n\
4	1	{}\n\
4	1	{7,8}\n\
5	2	{}\n\
5	INF	{9,10}\n\
6	2	{}\n\
6	INF	{}\n\
7	2	{}\n\
7	INF	{}\n\
8	2	{}\n\
8	INF	{}\n\
9	INF	{}\n\
9	INF	{}'

In [24]:
print(input_)

1	0	{2,3,4}
10	INF	{}
10	INF	{}
2	1	{}
2	1	{5,6}
3	1	{}
3	1	{}
4	1	{}
4	1	{7,8}
5	2	{}
5	INF	{9,10}
6	2	{}
6	INF	{}
7	2	{}
7	INF	{}
8	2	{}
8	INF	{}
9	INF	{}
9	INF	{}


In [61]:
class ReducerGraph():
    def reduce(self, v, values):
        d_min, childs = float('inf'), '{}'
        for d, struct in values:
            d = float('inf') if d == 'INF' else int(d)  # string to number
            if d < d_min:
                d_min = d
            if struct != '{}':
                childs = struct
        d_min = 'INF' if d_min == float('inf') else d_min
        print(v, d_min, childs, sep='\t')

In [62]:
reducer = ReducerGraph()
last_v, values = None, []
for line in input_.strip().split('\n'):
    v, d, childs = line.strip().split('\t')
    if last_v and v != last_v:
        reducer.reduce(last_v, values)
        last_v, values = v, []
        values.append((d, childs))
    else:
        last_v = v
        values.append((d, childs))
if last_v:
    reducer.reduce(last_v, values)

1	0	{2,3,4}
10	INF	{}
2	1	{5,6}
3	1	{}
4	1	{7,8}
5	2	{9,10}
6	2	{}
7	2	{}
8	2	{}
9	INF	{}
